# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [2]:
import os
import sqlite3
import pandas as pd   

In [3]:
from pathlib import Path

In [4]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [5]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()


## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [6]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [6]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [7]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [8]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [9]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [22]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  
*A compléter*

Les avantages

l'utilisation des vues dans SQL donne un accès limité aux utilisateurs dans le cas de données sensibles, de données complexes ou de grande quantités de données.
Cela ajoute une couche de sécurité supplémentaire à la base de données.
    
Désavantages:

Quand les tables sont souvent modifiées la vue doit être également modifiée.
l'utilisation de la vue ralentit les performances des requêtes.

https://fr.photo-555.com/5751867-sql-views


---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [7]:
# creation vue albums avec artistes et tracks
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist,
        nb_pistes,
        Durée_totale_Album_min,
        prixAlbum)
    AS
        SELECT albums.AlbumId,
               albums.Title,
               a.Name AS Artist,
               count(distinct tracks.TrackId) as nb_pistes,
               sum(tracks.Milliseconds)/60000 as Durée_totale_Album_min,
               sum(tracks.UnitPrice) as prixAlbum
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        JOIN tracks ON albums.AlbumId=tracks.AlbumId
        Group BY Artist, albums.Title
"""

executer_requete(creation_vue_albums, conn)

In [8]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist,nb_pistes,Durée_totale_Album_min,prixAlbum
0,1,For Those About To Rock We Salute You,AC/DC,10,40,9.90
1,4,Let There Be Rock,AC/DC,8,40,7.92
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1,3,0.99
3,267,Worlds,Aaron Goldberg,1,4,0.99
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2,7,1.98


In [16]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

In [25]:
# creation vue tracks avec tracks, albums, artistes, genres et mediatype
creation_vue_tracks = """
    CREATE VIEW v_tracks (
        TrackId,
        Name,
        Genre,
        Album,
        Artist,
        composer,
        MediaType, 
        Temps_en_seconde)
    AS
        SELECT  
    tracks.TrackId,
    tracks.Name,
    genres.Name,
    albums.Title,
    artists.Name,
    tracks.Composer as Compositeurs,
    media_types.Name as Nom_du_media,
    tracks.Milliseconds/1000 as durée_Album_en_sec
    FROM tracks
    JOIN genres ON  genres.GenreId=tracks.GenreId
    JOIN albums ON albums.AlbumId=tracks.AlbumId
    JOIN media_types ON media_types.MediaTypeId=tracks.MediaTypeId
    JOIN artists ON artists.ArtistId=albums.ArtistId  
    order by artists.Name
     
"""

executer_requete(creation_vue_tracks, conn)

In [26]:
# Utilisation de la vue v_tracks
tracks = pd.read_sql_query("SELECT * FROM v_tracks LIMIT 10;", conn)
tracks.head(5)

,TrackId,Name,Genre,Album,Artist,composer,MediaType,Temps_en_seconde
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",MPEG audio file,343
1,6,Put The Finger On You,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",MPEG audio file,205
2,7,Let's Get It Up,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",MPEG audio file,233
3,8,Inject The Venom,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",MPEG audio file,210
4,9,Snowballed,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",MPEG audio file,203


### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [37]:
# creation vue invoices
creation_vue_invoices = """
    CREATE VIEW v_invoices (
        InvoiceId,
        CustomerId,
        InvoiceDate,
        BillingAddress,
        BillingCity,
        BillingState,
        BillingCountry,
        BillingPostalCode,
        Total,
        semaine_transaction,
        mois_transaction,
        année_transaction)
    AS  
    SELECT
	*, STRFTIME("%W", invoices.InvoiceDate)+1 as semaine_transaction,  STRFTIME("%m", invoices.InvoiceDate) as mois_transaction,  STRFTIME("%Y", invoices.InvoiceDate) as année_transaction
FROM
	invoices 
     
"""

executer_requete(creation_vue_invoices, conn)

In [71]:
# Utilisation de la vue v_invoices
invoices = pd.read_sql_query("SELECT * FROM v_invoices LIMIT 10;", conn)
invoices.head(5)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,semaine_transaction,mois_transaction,année_transaction
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,01,2009
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,1,01,2009
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94,1,01,2009
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91,2,01,2009
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86,2,01,2009


In [36]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_invoices;", conn)

B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

In [72]:
# Utilisation de la vue v_invoices
invoices = pd.read_sql_query("SELECT sum(Total) as ventes, année_transaction, BillingCountry FROM v_invoices Group by année_transaction, BillingCountry order by année_transaction ;", conn)
invoices.head(25)

,ventes,année_transaction,BillingCountry
0,11.88,2009,Australia
1,1.98,2009,Austria
2,6.93,2009,Belgium
3,37.62,2009,Brazil
4,57.42,2009,Canada
5,15.84,2009,Chile
6,10.89,2009,Czech Republic
7,5.94,2009,Denmark
8,8.91,2009,Finland
9,35.64,2009,France


### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  

In [42]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_genres;", conn)

In [43]:
 creation_vue_genres = """
    CREATE VIEW v_genres (
        GenreId,
        Genres,
        nombre_pistes,
        Album,
        Artists,
        durée_Album,
        Prix)
    AS  
    SELECT genres.GenreId, 
    genres.Name as Nom_Genre, 
    count(distinct tracks.TrackId) as nombre_pistes, 
    albums.Title as Album,
    artists.Name as Artists,
    sum(tracks.Milliseconds)/60000 as durée_Album_en_min, 
    sum(tracks.UnitPrice) as coût_total
    FROM tracks
    JOIN genres ON  genres.GenreId=tracks.GenreId
    JOIN albums ON albums.AlbumId=tracks.AlbumId
    JOIN artists ON artists.ArtistId=albums.ArtistId
   Group by Album
    Order by genres.Name
    
    """

executer_requete(creation_vue_genres, conn)

In [45]:
# Utilisation de la vue v_genres
genres = pd.read_sql_query("SELECT * FROM v_genres;", conn)
genres.head(20)

,GenreId,Genres,nombre_pistes,Album,Artists,durée_Album,Prix
0,23,Alternative,1,Cake: B-Sides and Rarities,Cake,3,0.99
1,23,Alternative,1,Carried to Dust (Bonus Track Version),Calexico,3,0.99
2,23,Alternative,14,Carry On,Chris Cornell,54,13.86
3,23,Alternative,14,Revelations,Audioslave,58,13.86
4,23,Alternative,10,Temple of the Dog,Temple of the Dog,55,9.90
5,4,Alternative & Punk,22,Acústico,Titãs,71,21.78
6,4,Alternative & Punk,12,Album Of The Year,Faith No More,43,11.88
7,4,Alternative & Punk,13,American Idiot,Green Day,57,12.87
8,4,Alternative & Punk,14,Angel Dust,Faith No More,61,13.86
9,4,Alternative & Punk,17,Blood Sugar Sex Magik,Red Hot Chili Peppers,73,16.83
